### Создание модели и обучение ее на одной выборке 

In [1]:
!python --version

Python 3.9.13


In [2]:
import numpy as np 
import time
import random
import os

from tensorflow.keras.models import Model       
from tensorflow.keras.layers import Input, Conv2DTranspose, concatenate, Activation, MaxPooling2D, Conv2D, BatchNormalization
from tensorflow.keras.layers import Reshape, GlobalAveragePooling2D, UpSampling2D, AveragePooling2D
from tensorflow.keras import backend as K        
from tensorflow.keras.optimizers import Adam     
from tensorflow.keras import utils               
from tensorflow.keras.preprocessing import image 
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt  
# from PIL import Image 

In [3]:
SEED = 1111
random.seed(SEED)
np.random.seed(SEED)

import tensorflow as tf
tf.random.set_seed(SEED)

In [4]:
# Путь к сохраненным выборкам
data_path = "/home/serg/bin/NIIAS-RZD/data_np"
# Путь для сохранения моделей
model_path = "/home/serg/bin/NIIAS-RZD/models"

In [5]:
np_files= [ os.path.join(data_path, file_name) for file_name in sorted(os.listdir(data_path))]
print("Имеем {} сохраненных выборок".format(len(np_files)))

Имеем 40 сохраненных выборок


In [ ]:
# Загрузим сохраненные выборки с конкатенацией массивов
# НА НОУТЕ НЕ ХВАТАЕТ ПАМЯТИ
start = 0
stop = 5

first_file = np_files[start]
files_to_load = np_files[start+1:stop]

print(first_file)
data = np.load(first_file)
images_np = data['a']
anns_np = data['b']
print(images_np.shape, anns_np.shape)

for file_name in files_to_load:
    print(file_name)
    data = np.load(file_name)
    curr_images_np = data['a']
    curr_anns_np = data['b']
    
    images_np = np.concatenate([images_np, curr_images_np])
    anns_np = np.concatenate([anns_np, curr_anns_np])
    
    print(images_np.shape, anns_np.shape)

(200, 432, 768, 3) (200, 432, 768, 4)
/home/serg/bin/NIIAS-RZD/data_np/data10.npz
(400, 432, 768, 3) (400, 432, 768, 4)
/home/serg/bin/NIIAS-RZD/data_np/data11.npz


In [13]:
# Загрузим сохраненную выборку
np_file = random.choice(np_files)
print(np_file)
data = np.load(np_file)
images_np = data['a']
anns_np = data['b']
print(images_np.shape, anns_np.shape)

/home/serg/bin/NIIAS-RZD/data_np/data18.npz
(200, 432, 768, 3) (200, 432, 768, 4)


In [14]:
# Разделим train_test_split
x_train, x_val, y_train, y_val = train_test_split(images_np, anns_np, test_size=0.15, shuffle=True)    
print (x_train.shape)
print (y_train.shape)
print (x_val.shape)
print (y_val.shape)

(170, 432, 768, 3)
(170, 432, 768, 4)
(30, 432, 768, 3)
(30, 432, 768, 4)


In [17]:
# Размеры, изображения
img_height = x_train.shape[1]      # 432
img_width = x_train.shape[2]       # 768
print(img_height, img_width)

432 768


In [20]:
# Классов сегментации
classes = [0, 6, 7, 10]
num_classes = len(classes)
print(num_classes)

4


### Функции

In [22]:
# Функция метрики, обрабатывающая пересечение двух областей
def dice_coef(y_true, y_pred):
  # Возвращаем площадь пересечения деленную на площадь объединения двух областей
  return (2. * K.sum(y_true * y_pred) + 1.) / (K.sum(y_true) + K.sum(y_pred) + 1.) 

### Создание PSP-net

Создадим модель, попробуем обучить и посмотреть результаты

<img src="./PSPNet.png" width=800 heigth=480>

In [15]:
'''
  Функция создания сети
    Входные параметры:
    - num_classes - количество классов
    - input_shape - размерность карты сегментации
'''
def pspnet(num_classes = 4, input_shape= (432, 768, 3)):
    img_input = Input(input_shape)                                         # Создаем входной слой с размерностью input_shape

    # Feature Map
    x = Conv2D(64, (3, 3), padding='same', name='feature_map_conv1')(img_input)  # Добавляем Conv2D-слой с 64-нейронами
    x = BatchNormalization()(x)                                                  # Добавляем слой BatchNormalization
    x = Activation('relu')(x)                                                    # Добавляем слой Activation

    x = Conv2D(128, (3, 3), padding='same', name='feature_map_conv2')(x)         # Добавляем Conv2D-слой с 64-нейронами
    x = BatchNormalization()(x)                                                  # Добавляем слой BatchNormalization
    x = Activation('relu')(x)                                                    # Добавляем слой Activation
 
    x = Conv2D(256, (3, 3), padding='same', name='feature_map_conv3')(x)         # Добавляем Conv2D-слой с 64-нейронами
    x = BatchNormalization()(x)                                                  # Добавляем слой BatchNormalization
    feature_map_out = Activation('relu', name='feature_map_out')(x)              # Добавляем слой Activation 
 

    # Pyramyd Pooling Module
    # red
    #red = GlobalAveragePooling2D(name='red_pool')(feature_map_out)
    #red = Reshape((1,1,256))(red)
    red = AveragePooling2D(pool_size=(1,1),name='red_pool')(feature_map_out)
    red = Conv2D(filters=64,kernel_size=(1,1),name='red_conv')(red)
    #red = UpSampling2D(size=256,interpolation='bilinear',name='red_upsampling')(red)
    red = UpSampling2D(size=1,interpolation='bilinear',name='red_upsampling')(red)

    # yellow
    yellow = AveragePooling2D(pool_size=(2,2),name='yellow_pool')(feature_map_out)
    yellow = Conv2D(filters=64,kernel_size=(1,1),name='yellow_conv')(yellow)
    yellow = UpSampling2D(size=2,interpolation='bilinear',name='yellow_upsampling')(yellow)

    # blue
    blue = AveragePooling2D(pool_size=(4,4),name='blue_pool')(feature_map_out)
    blue = Conv2D(filters=64,kernel_size=(1,1),name='blue_conv')(blue)
    blue = UpSampling2D(size=4,interpolation='bilinear',name='blue_upsampling')(blue)

    # green
    green = AveragePooling2D(pool_size=(8,8),name='green_pool')(feature_map_out)
    green = Conv2D(filters=64,kernel_size=(1,1),name='green_conv')(green)
    green = UpSampling2D(size=8,interpolation='bilinear',name='green_upsampling')(green)

    # feature_map + red + yellow + blue + green
    pooling_module_out = concatenate([feature_map_out,red,yellow,blue,green])


    # Final Prediction
    x = Conv2D(64, (3, 3), padding='same')(pooling_module_out)                   # Добавляем слой Conv2D с 64 нейронами
    x = BatchNormalization()(x)                                                  # Добавляем слой BatchNormalization
    x = Activation('relu')(x)                                                    # Добавляем слой Activation

    x = Conv2D(num_classes, (3, 3), activation='softmax', padding='same')(x)     # Добавляем Conv2D-Слой с softmax-активацией на num_classes-нейронов

    model = Model(img_input, x) # Создаем модель с входом 'img_input' и выходом 'x'

    # Компилируем модель 
    model.compile(optimizer=Adam(),
                  loss='categorical_crossentropy',
                  metrics=[dice_coef])
    
    return model # Возвращаем сформированную модель

In [26]:
# Создаем модель
modelPSPnet = pspnet(num_classes, (img_height, img_width, 3))

In [27]:
modelPSPnet.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 432, 768, 3  0           []                               
                                )]                                                                
                                                                                                  
 feature_map_conv1 (Conv2D)     (None, 432, 768, 64  1792        ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_8 (BatchNo  (None, 432, 768, 64  256        ['feature_map_conv1[0][0]']      
 rmalization)                   )                                                           

                                                                                                  
Total params: 735,940
Trainable params: 734,916
Non-trainable params: 1,024
__________________________________________________________________________________________________


In [32]:
# Обучаем модель
history = modelPSPnet.fit(x_train, y_train, epochs=10, batch_size=1, validation_data = (x_val, y_val)) 

Epoch 1/10


2022-07-18 23:03:05.731307: W tensorflow/core/common_runtime/bfc_allocator.cc:479] Allocator (GPU_0_bfc) ran out of memory trying to allocate 324.00MiB (rounded to 339738624)requested by op Adam/gradients/AddN/tmp_var_zeros
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2022-07-18 23:03:05.732023: I tensorflow/core/common_runtime/bfc_allocator.cc:1027] BFCAllocator dump for GPU_0_bfc
2022-07-18 23:03:05.732092: I tensorflow/core/common_runtime/bfc_allocator.cc:1034] Bin (256): 	Total Chunks: 103, Chunks in use: 103. 25.8KiB allocated for chunks. 25.8KiB in use in bin. 15.7KiB client-requested in use in bin.
2022-07-18 23:03:05.732120: I tensorflow/core/common_runtime/bfc_allocator.cc:1034] Bin (512): 	Total Chunks: 21, Chunks in use: 21. 10.5KiB allocated for chunks. 10.5KiB in use in bin. 10.5KiB client-requested in use in bin.


2022-07-18 23:03:05.735762: I tensorflow/core/common_runtime/bfc_allocator.cc:1083] InUse at 501269000 of size 256 next 166
2022-07-18 23:03:05.735774: I tensorflow/core/common_runtime/bfc_allocator.cc:1083] InUse at 501269100 of size 256 next 167
2022-07-18 23:03:05.735787: I tensorflow/core/common_runtime/bfc_allocator.cc:1083] InUse at 501269200 of size 512 next 168
2022-07-18 23:03:05.735801: I tensorflow/core/common_runtime/bfc_allocator.cc:1083] InUse at 501269400 of size 512 next 169
2022-07-18 23:03:05.735814: I tensorflow/core/common_runtime/bfc_allocator.cc:1083] InUse at 501269600 of size 512 next 170
2022-07-18 23:03:05.735826: I tensorflow/core/common_runtime/bfc_allocator.cc:1083] InUse at 501269800 of size 1024 next 172
2022-07-18 23:03:05.735838: I tensorflow/core/common_runtime/bfc_allocator.cc:1083] InUse at 501269c00 of size 1280 next 91
2022-07-18 23:03:05.735851: I tensorflow/core/common_runtime/bfc_allocator.cc:1083] InUse at 50126a100 of size 9216 next 90
2022-07

ResourceExhaustedError: Graph execution error:

OOM when allocating tensor of shape [1,256,432,768] and type float
	 [[{{node Adam/gradients/AddN/tmp_var_zeros}}]] [Op:__inference_train_function_2848]